In [108]:
import pandas as pd
import numpy as np
import math
from sklearn.cluster import KMeans

In [91]:
df_MRN_h = pd.read_csv("data/CDEC/MRN/ytd_hr_humidity.csv")
df_MRN_h

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,MRN,H,12,REL HUM,20240101 0000,20240101 0000,94,,%
1,MRN,H,12,REL HUM,20240101 0100,20240101 0100,96,,%
2,MRN,H,12,REL HUM,20240101 0200,20240101 0200,89,,%
3,MRN,H,12,REL HUM,20240101 0300,20240101 0300,92,,%
4,MRN,H,12,REL HUM,20240101 0400,20240101 0400,92,,%
...,...,...,...,...,...,...,...,...,...
2635,MRN,H,12,REL HUM,20240419 2000,20240419 2000,62,,%
2636,MRN,H,12,REL HUM,20240419 2100,20240419 2100,68,,%
2637,MRN,H,12,REL HUM,20240419 2200,20240419 2200,72,,%
2638,MRN,H,12,REL HUM,20240419 2300,20240419 2300,79,,%


In [92]:
df_MRN_p = pd.read_csv("data/CDEC/MRN/ytd_hr_precipitation.csv")
df_MRN_p

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,MRN,H,2,RAIN,20240101 0000,20240101 0000,2.85,,INCHES
1,MRN,H,2,RAIN,20240101 0100,20240101 0100,2.85,,INCHES
2,MRN,H,2,RAIN,20240101 0200,20240101 0200,2.85,,INCHES
3,MRN,H,2,RAIN,20240101 0300,20240101 0300,2.85,,INCHES
4,MRN,H,2,RAIN,20240101 0400,20240101 0400,2.85,,INCHES
...,...,...,...,...,...,...,...,...,...
2635,MRN,H,2,RAIN,20240419 2000,20240419 2000,15.48,,INCHES
2636,MRN,H,2,RAIN,20240419 2100,20240419 2100,15.48,,INCHES
2637,MRN,H,2,RAIN,20240419 2200,20240419 2200,15.48,,INCHES
2638,MRN,H,2,RAIN,20240419 2300,20240419 2300,15.48,,INCHES


In [93]:
df_MRN_t = pd.read_csv("data/CDEC/MRN/ytd_hr_temp.csv")
df_MRN_t

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F
...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F


In [94]:
# every temperature point gets assigned to its closest station
# MRN long lat 34.317, -117.502
df_MRN_h = df_MRN_h.rename(columns={'VALUE': 'REL_HUM_VALUE'})
df_MRN_p = df_MRN_p.rename(columns={'VALUE': 'RAIN_VALUE'})
df_MRN_t = df_MRN_t.rename(columns={'VALUE': 'TEMP_VALUE'})

df_MRN_t['REL_HUM_VALUE'] = df_MRN_h.get("REL_HUM_VALUE")
df_MRN_t['RAIN_VALUE'] = df_MRN_p.get("RAIN_VALUE")
df_MRN_t

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,TEMP_VALUE,DATA_FLAG,UNITS,REL_HUM_VALUE,RAIN_VALUE
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F,94,2.85
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F,96,2.85
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F,89,2.85
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F,92,2.85
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F,92,2.85
...,...,...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F,62,15.48
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F,68,15.48
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F,72,15.48
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F,79,15.48


In [95]:
df_MRN_t.dropna()

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,TEMP_VALUE,DATA_FLAG,UNITS,REL_HUM_VALUE,RAIN_VALUE
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F,94,2.85
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F,96,2.85
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F,89,2.85
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F,92,2.85
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F,92,2.85
...,...,...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F,62,15.48
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F,68,15.48
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F,72,15.48
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F,79,15.48


In [96]:
def t_c(temp):
    if temp == "---":
        return 0
    else:
        return temp
df_MRN_t['TEMP_VALUE'] = df_MRN_t.get("TEMP_VALUE").apply(t_c)
df_MRN_t

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,TEMP_VALUE,DATA_FLAG,UNITS,REL_HUM_VALUE,RAIN_VALUE
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F,94,2.85
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F,96,2.85
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F,89,2.85
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F,92,2.85
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F,92,2.85
...,...,...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F,62,15.48
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F,68,15.48
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F,72,15.48
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F,79,15.48


In [106]:
def simplified_cip_model(station_data, cloud_top_temperature=-27, cloud_mask=0.8):
    """
    Applies a simplified CIP model to weather station data with constant cloud top temperature and cloud mask.

    Args:
        station_data (pandas.DataFrame): DataFrame containing temperature, humidity, and rain values.
        cloud_top_temperature (float): Constant cloud top temperature in degrees Fahrenheit (default: 70).
        cloud_mask (float): Constant cloud mask value (default: 0.8).

    Returns:
        pandas.DataFrame: DataFrame with an added "res" column containing icing potential percentages.
    """

    # --- Data Preparation ---
    # Ensure temperature is in Celsius
    def sigmoid(x):
        return 1 / (1 + math.exp(-x))
    
    station_data['TEMP_VALUE_C'] = (pd.to_numeric(station_data['TEMP_VALUE']) - 32) * 5/9
    #cloud_m = 
    # --- Fuzzy Logic Membership Functions ---
    def tmap(temperature):
        # Example Tmap implementation (adjust ranges and logic as needed)
        if temperature < 30:
            return 0.0
        elif 30 <= temperature <= 45:
            return (temperature + 25) / 10
        elif 45 < temperature <= 50:
            return 1.0
        elif 50 < temperature <= 60:
            return (-temperature) / 3
        else:
            return 0.0

    def cttmap(ctt):
        # Example CTTmap implementation (adjust ranges and logic as needed)
        if ctt < -30:
            return 0.1
        elif -30 <= ctt <= -12:
            return (ctt + 30) / 18
        elif -12 < ctt <= 8:
            return 1.0
        elif 8 < ctt <= 12:
            return (12 - ctt) / 4
        else:
            return 0.5

    def rhmap(rh):
        # Example RHmap implementation 
        if rh < 70:
            return 0.0
        elif 70 <= rh <= 90:
            return (rh - 70) / 20
        else:
            return 1.0
            return 1.0

    # --- Icing Potential Calculation ---
    
    
    def calculate_icing_potential(row):
        if row['TEMP_VALUE_C'] == "---":
            temperature = 0
        else:
            temperature = int(row['TEMP_VALUE'])
        if row['RAIN_VALUE'] == "---":
            rain = 0
        else:
            rain = float(row['RAIN_VALUE'])
        if row['REL_HUM_VALUE'] == "---":
            humidity = 0
        else:
            humidity = int(row['REL_HUM_VALUE'])
        #humidity = row['REL_HUM_VALUE']
        print(type(temperature))
        print(temperature)
        print(type(rain))
        print(rain)
        print(type(humidity))
        print(humidity)
        
        if cloud_mask >= 0.5 and rain > 0:  
            if temperature >= 0:  
                return sigmoid(tmap(temperature) * cttmap(cloud_top_temperature))
            else:  
                return sigmoid(0.8 * tmap(temperature) * cttmap(cloud_top_temperature))
        elif cloud_mask >= 0.5:  # Assuming cloud presence based on cloud mask threshold
            return sigmoid(tmap(temperature) * cttmap(cloud_top_temperature) * rhmap(humidity))
        else:
            return 0.0

    # --- Apply Model and Add Results ---
    station_data['res'] = station_data.apply(calculate_icing_potential, axis=1)

    # Convert icing potential to percentage
    station_data['res'] = station_data['res'] * 100

    return station_data

In [107]:
simplified_cip_model(df_MRN_t)

<class 'int'>
36
<class 'float'>
2.85
<class 'int'>
94
<class 'int'>
35
<class 'float'>
2.85
<class 'int'>
96
<class 'int'>
38
<class 'float'>
2.85
<class 'int'>
89
<class 'int'>
36
<class 'float'>
2.85
<class 'int'>
92
<class 'int'>
35
<class 'float'>
2.85
<class 'int'>
92
<class 'int'>
38
<class 'float'>
2.85
<class 'int'>
86
<class 'int'>
38
<class 'float'>
2.85
<class 'int'>
85
<class 'int'>
39
<class 'float'>
2.85
<class 'int'>
85
<class 'int'>
42
<class 'float'>
2.85
<class 'int'>
80
<class 'int'>
43
<class 'float'>
2.85
<class 'int'>
75
<class 'int'>
46
<class 'float'>
2.85
<class 'int'>
69
<class 'int'>
50
<class 'float'>
2.85
<class 'int'>
63
<class 'int'>
51
<class 'float'>
2.85
<class 'int'>
60
<class 'int'>
53
<class 'float'>
2.85
<class 'int'>
52
<class 'int'>
52
<class 'float'>
2.85
<class 'int'>
49
<class 'int'>
53
<class 'float'>
2.85
<class 'int'>
50
<class 'int'>
47
<class 'float'>
2.85
<class 'int'>
60
<class 'int'>
45
<class 'float'>
2.85
<class 'int'>
64
<class 'in

,STATION_ID,DURATION,SENSOR_NUMBER,SENSOR_TYPE,DATE TIME,OBS DATE,TEMP_VALUE,DATA_FLAG,UNITS,REL_HUM_VALUE,RAIN_VALUE,TEMP_VALUE_C,res
0,MRN,H,4,TEMP,20240101 0000,20240101 0000,36,,DEG F,94,2.85,2.222222,73.432280
1,MRN,H,4,TEMP,20240101 0100,20240101 0100,35,,DEG F,96,2.85,1.666667,73.105858
2,MRN,H,4,TEMP,20240101 0200,20240101 0200,38,,DEG F,89,2.85,3.333333,74.077490
3,MRN,H,4,TEMP,20240101 0300,20240101 0300,36,,DEG F,92,2.85,2.222222,73.432280
4,MRN,H,4,TEMP,20240101 0400,20240101 0400,35,,DEG F,92,2.85,1.666667,73.105858
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,MRN,H,4,TEMP,20240419 2000,20240419 2000,55,,DEG F,62,15.48,12.777778,4.497823
2636,MRN,H,4,TEMP,20240419 2100,20240419 2100,52,,DEG F,68,15.48,11.111111,5.270557
2637,MRN,H,4,TEMP,20240419 2200,20240419 2200,50,,DEG F,72,15.48,10.000000,54.157048
2638,MRN,H,4,TEMP,20240419 2300,20240419 2300,48,,DEG F,79,15.48,8.888889,54.157048


In [29]:
def testing(row):
    print(row["RAIN_VALUE"])

test = df_MRN_t.apply(testing, axis = 1)
test

2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.85
2.86
2.87
3.00
3.00
3.00
---
3.00
---
---
---
---
---
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3.00
3

0       None
1       None
2       None
3       None
4       None
        ... 
2635    None
2636    None
2637    None
2638    None
2639    None
Length: 2640, dtype: object

In [112]:
X = np.array([[33, -110], [34, 118], [34, -118.01],
             [23, 65], [20, 45], [15, 40]])

ks = np.array([[34.317, -117.502], [34,118.2],[33.984, -118.01],[20,50]])
def euclidean_distance(array1, array2):

    if array1.shape != array2.shape:
        raise ValueError("Arrays must have the same shape.")
    squared_diffs = np.square(array1 - array2)
    sum_squared_diffs = np.sum(squared_diffs)
    distance = np.sqrt(sum_squared_diffs)

    return distance

ans = []
for x in X:
    dist = [Inf,0]
    for j,k in enumerate(ks):
        dist_s = euclidean_distance(k, x)
        if dist_s < dist[0]:
            dist[0] = dist_s
            dist[1] = j
    ans.append(dist[1])
ans

    
        
        
        
        

[0, 1, 2, 3, 3, 3]

In [ ]:
column for neighbors stores lists []
each list has its respective value